# VSB Powerline rgb 

using spectrograms created in vsb_powerline_spectrograms

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import datetime
import uuid
from os import listdir
from os.path import isfile, join
import glob

In [3]:
from fastai.vision import *

In [4]:
from fastai import *
from fastai.tabular import *
from fastai.utils import *

In [5]:
import fastai
print(fastai.__version__)

1.0.39.dev0


In [6]:
torch.cuda.set_device(0)

In [7]:
DATE = datetime.datetime.today().strftime('%Y%m%d')
UID=str(uuid.uuid4())[:8]

In [8]:
DATE

'20181229'

In [9]:
UID

'65876233'

In [10]:
image_size=300
bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

In [11]:
path = Path('../input/')
#train_path = path/'train/'
#train_rgb = path/'train_rgb/'
train_path = path/'train_300_-110-30/'
train_rgb = path/'train_rgb_300_-110-30/'

In [12]:
#test_path = path/'test/'
#test_rgb = path/'test_rgb/'
test_path = path/'test_300_-110-30/'
test_rgb = path/'test_rgb_300_-110-30/'

In [13]:
samples = 800000

In [14]:
total_time_sec = 20/1000
sr = total_time_sec/samples

In [15]:
def read_meta():
    meta_train = pd.read_csv(path/'metadata_train.csv')
    features = meta_train.columns
    meta_test = pd.read_csv(path/'metadata_test.csv')
    return meta_train, features, meta_test

In [16]:
meta_train, features, meta_test=read_meta()

In [17]:
meta_train.tail()

,signal_id,id_measurement,phase,target
8707,8707,2902,1,0
8708,8708,2902,2,0
8709,8709,2903,0,0
8710,8710,2903,1,0
8711,8711,2903,2,0


In [18]:
### Parquet Data

In [19]:
def read_parquet():
    start_time = time.time()
    train_df = pq.read_pandas(path/'train.parquet').to_pandas()
    end_time = time.time()
    print(f'loading took {end_time-start_time} secs')
    train_df = train_df.T
    print(f'train_df.shape: {train_df.shape}')

    start_time = time.time()
    test_df = pq.read_pandas(path/'test.parquet').to_pandas()
    end_time = time.time()
    print(f'loading took {end_time-start_time} secs')
    test_df = test_df.T
    print(f'test_df.shape: {test_df.shape}')
    
    return train_df, test_df

In [20]:
#train_df, test_df=read_parquet()

In [21]:
#get first channel
#y=train_df.iloc[0, :]

In [22]:
#len(y)

In [23]:
def gen_rgb(img_r, img_g, img_b, id_m, img_path, out_path):
    red = np.array(PIL.Image.open(img_path/f'{img_r}.jpg').convert("L"))
    green = np.array(PIL.Image.open(img_path/f'{img_g}.jpg').convert("L"))
    blue = np.array(PIL.Image.open(img_path/f'{img_b}.jpg').convert("L"))
    rgb=PIL.Image.fromarray(np.concatenate((np.expand_dims(red,axis=2),np.expand_dims(green,axis=2),np.expand_dims(blue,axis=2)),axis=2))
    rgb_file = f'{out_path}/{id_m}_rgb.png'
    rgb.save(rgb_file)
    rgb.close()
    

In [24]:
def channelize_images(df, img_path, out_path):
    print('>>channelize_images')
    #phase_0 = df.iloc[::0, :]
    i=0
    for index, row in df.iterrows():
        if i==0:
            assert row["phase"]==0
            red = row['signal_id']
        elif i==1:
            assert row["phase"]==1
            green = row['signal_id']
        elif i==2:
            assert row["phase"]==2
            blue = row['signal_id']
            id_m = row['id_measurement']
            #should be sequential signals
            assert int(blue)==int(green)+1
            assert int(green)==int(red)+1
            gen_rgb(red, green, blue, id_m, img_path, out_path)
            red=None
            green=None
            blue=None
            id_m=None
            i=-1
        i+=1
    

In [25]:
channelize_images(meta_train, train_path, train_rgb)

>>channelize_images
